In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import nan_euclidean_distances

In [176]:
df = pd.read_csv('/Users/andrescrucettanieto/Documents/GitHub/missing_data_project/data/toy_data.csv')

In [177]:
def kSmallest(arr, k):
    
    # Sort the given array 
    arr_i = arr.copy()
    arr_i.sort()
  
    # Return k'th element in the sorted array 
    # Skips the first element since it contains 
    # the value 0
    return arr_i[1:k+1]

In [185]:
def normalize_data(df):
    '''
    Normalizes the data in X
    '''
    scaler = MinMaxScaler()
    df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)
    return df

In [142]:
array_column = np.array([[1],[5],[3],[4],[5],[6],[1],[8],[3],[2]])
ksmallest = kSmallest(array_column, 3)

In [112]:
ksmallest

array([1, 2, 3])

In [113]:
np.where(array_column == ksmallest)[0]

array([0, 2, 6, 8, 9])

In [186]:
numeric_df = df.select_dtypes(include=[np.number])

In [190]:
numeric_df
numeric_df =  normalize_data(numeric_df)

In [191]:
euclidean_row = numeric_df.iloc[0]
euclidean_row

Numerical A       0.040629
Numerical B       0.296947
Output Value 1    0.000000
Name: 0, dtype: float64

In [192]:
k = 3

for j in range(len(numeric_df.columns)):
    lst_missing = numeric_df.iloc[:,j][numeric_df.iloc[:,j].isnull()].index.to_list()
    print(lst_missing)
    for i in lst_missing:
        # Get the comparison row
        euclidean_row = numeric_df.iloc[i]
        
        # Take euclidean distances of the other columns
        euclidean_distances = nan_euclidean_distances(numeric_df, [euclidean_row.to_list()])
        
        # Flatten the euclidean distance array
        euc_flattened = euclidean_distances.flatten(order='F')
        
        # Get the k smallest distances
        k_nearest = kSmallest(euc_flattened, k)
        k_nearest_indices = np.where(np.in1d(euc_flattened, k_nearest))[0]
        
        # Get mean of the k nearest neighbors
        k_mean = numeric_df.iloc[k_nearest_indices].iloc[:,j].mean()
        
        # Assigning the value to the nan row cell
        print(i,j)
        numeric_df.iloc[i].iloc[j]=k_mean

[2, 6]
2 0
6 0
[4]
4 1
[2, 5]
2 2
5 2


In [193]:
numeric_df

,Numerical A,Numerical B,Output Value 1
0,0.040629,0.296947,0.000000
1,0.000000,0.489844,0.046875
2,0.346876,0.280872,0.304688
3,0.314548,0.665919,0.585938
4,0.716907,0.329429,0.035156
5,1.000000,0.182928,0.304688
6,0.553808,0.000000,0.878906
7,0.730013,0.524486,0.000000
8,0.828309,1.000000,1.000000


In [151]:
vals = numeric_df.iloc[k_nearest_indices].iloc[:,0].mean()

In [155]:
numeric_df.iloc[2].iloc[0]=vals

In [156]:
numeric_df

,Numerical A,Numerical B,Output Value 1
0,1.130000,26.96,1.65
1,0.820000,42.44,1.77
2,3.466667,25.67,NaN
3,3.220000,56.57,3.15
4,6.290000,NaN,1.74
5,8.450000,17.81,NaN
6,NaN,3.13,3.90
7,6.390000,45.22,1.65
8,7.140000,83.38,4.21


In [ ]:
# Iterate over the other columns and calculate the euclidean distance

# Choose the k nearest neighbors to the value

# Average out the values and impute the missing value